In [1]:
import sys
sys.path.append("./")
import time
import pickle
import os
import random
import openpyxl
import argparse
import copy
import pandas as pd

from get_fos import GET_FOS
from result import RESULT
from pre_set import PRE_SET
from digest import DIGEST
from get_structure import GET_STRUCTURE
from initialization import INITIAL
from first_algo import FIRST_ALGO
from second_algo import SECOND_ALGO
from resonance import RESONANCE
from tune import TUNE
from post_process import *
from auxilary import *

In [2]:
path = os.path.dirname(os.getcwd())

file_get = open(os.path.join(path,"valid_t_dict.pkl"),'rb') 
valid_t_dict = pickle.load(file_get) 
file_get.close()


file_get= open(os.path.join(path,"global_normalized_normed_dict.pkl"),"rb")
global_normalized_normed_dict = pickle.load(file_get)
file_get.close()

file_get= open(os.path.join(path,"global_mean_dict.pkl"),"rb")
global_mean_dict = pickle.load(file_get)
file_get.close()

file_get= open(os.path.join(path,"global_sigma_dict.pkl"),"rb")
global_sigma_dict = pickle.load(file_get)
file_get.close()

In [40]:
from post_process import *
import copy
import itertools
import math
import numpy as np
import random

class TUNE():
	def __init__(self):
		None

	def apply_resonance(self, atom_idx_list, valence_list, LOSS, res, global_nomalized_dict, 
																	  global_sigma_dict, 
																	  global_mean_dict):
		ori_LOSS = copy.deepcopy(LOSS)
		classes_dict = classify(atom_idx_list,res)
		valence_with_loss = []
		NC = len((classes_dict))
		print(NC)
		print("resonance_order:", res.resonance_order)
		timestamp = time.time()
		for c in classes_dict:
			for d in classes_dict:
				if c[0] != d[0]:
					op_c = classes_dict[c]
					op_d = classes_dict[d]

					now_valence_list_c = [valence_list[i] for i in op_c]
					now_max_oxi_list_c = [res.max_oxi_list[i] for i in op_c]
					now_min_oxi_list_c = [max(0,res.min_oxi_list[i]) for i in op_c]   #because they are the super atom!
					now_valence_list_d = [valence_list[i] for i in op_d]
					now_max_oxi_list_d = [res.max_oxi_list[i] for i in op_d]
					now_min_oxi_list_d = [max(0,res.min_oxi_list[i]) for i in op_d]   #same reason.

					if len(op_c) < len(op_d):
						now_valence_list_c = [i + 1 for i in now_valence_list_c]
						N = len(op_c)
						while N != 0:
							temp_list = sorted([(j, v) for j,v in enumerate(now_valence_list_d)], reverse = True, key = lambda x:x[1])
							now_valence_list_d[temp_list[0][0]] -= 1
							N -= 1

						target_valence = set(now_valence_list_d)

						#make sure the volume of the memory!
						total_combi = math.factorial(len(op_d))/math.factorial(len(op_d)-len(op_c))/math.factorial(len(op_c))
						while total_combi >= 10000000:
							op_d = random.sample(op_d, round(len(op_d)*(4/5)))
							total_combi = math.factorial(len(op_d))/math.factorial(len(op_d)-len(op_c))/math.factorial(len(op_c))
							print("47", total_combi, res.mid)

						possible_combi = []
						for p in itertools.combinations(op_d, len(op_c)):
							possible_combi.append(p)
						num_sample = round(5000/(NC**2)) if int(res.resonance_order) < 2 else round((5000)**(1/2)/(NC**2))
						if len(possible_combi) >= num_sample:
							possible_combi = random.sample(possible_combi, num_sample)

						resonance = {"op_d": possible_combi, "target_valence":target_valence}
						

					if len(op_c) > len(op_d):
						now_valence_list_d = [j - 1 for j in now_valence_list_d]
						N = len(op_d)
						while N != 0:
							temp_list = sorted([(i, v) for i,v in enumerate(now_valence_list_c)], reverse = False, key = lambda x:x[1])
							now_valence_list_c[temp_list[0][0]] += 1
							N -= 1
                        
						target_valence = set(now_valence_list_c)

						#make sure the volume of the memory!
						total_combi = math.factorial(len(op_c))/math.factorial(len(op_c)-len(op_d))/math.factorial(len(op_d))
						while total_combi >= 10000000:
							op_c = random.sample(op_c, round(len(op_c)*(4/5)))
							total_combi = math.factorial(len(op_c))/math.factorial(len(op_c)-len(op_d))/math.factorial(len(op_d))
							print("74", total_combi, res.mid)
						possible_combi = []

						for p in itertools.combinations(op_c, len(op_d)):
							possible_combi.append(p)
						num_sample = round(5000/(NC**2)) if int(res.resonance_order) < 2 else round((5000)**(1/2)/(NC**2))
						if len(possible_combi) >= num_sample:
							possible_combi = random.sample(possible_combi, num_sample)

						resonance = {"op_c": possible_combi,"target_valence":target_valence}
						
						print(num_sample)
					if len(op_c) == len(op_d):
						now_valence_list_c = [i + 1 for i in now_valence_list_c]
						now_valence_list_d = [j - 1 for j in now_valence_list_d]
						resonance = {"equal": [tuple(op_c)]}

					if all([now_max_oxi_list_c[i] >= now_valence_list_c[i] for i in range(len(op_c))]) and all(
							[now_min_oxi_list_d[i] <= now_valence_list_d[i] for i in range(len(op_d))]):

						resonance_result = []
						if list(resonance.keys())[0] == "op_d":
							possible_valence_list = []
							for p in resonance["op_d"]:
								temp_sum_of_valence = copy.deepcopy(valence_list)
								for i in op_c:
									temp_sum_of_valence[i] += 1
								for j in p:
									temp_sum_of_valence[j] -= 1
								if set([temp_sum_of_valence[i] for i in op_d]) == resonance["target_valence"]:
									possible_valence_list.append(temp_sum_of_valence)                                         

						if list(resonance.keys())[0] == "op_c":
							possible_valence_list = []
							for p in resonance["op_c"]:
								temp_sum_of_valence = copy.deepcopy(valence_list)
								for i in p:
									temp_sum_of_valence[i] += 1
								for j in op_d:
									temp_sum_of_valence[j] -= 1
								if set([temp_sum_of_valence[i] for i in op_c]) == resonance["target_valence"]:
									possible_valence_list.append(temp_sum_of_valence)

						if list(resonance.keys())[0] == "equal":
							temp_sum_of_valence = copy.deepcopy(valence_list)
							for i in op_c:
								temp_sum_of_valence[i] += 1
							for j in op_d:
								temp_sum_of_valence[j] -= 1
							possible_valence_list = [temp_sum_of_valence] 

						for vl in possible_valence_list:
							#print(len(possible_valence_list))
							temp_pair_info = spider_pair_length_with_CN_unnorm(vl, res)
							temp_LOSS = cal_loss_func_by_MAP(temp_pair_info,global_nomalized_dict, global_sigma_dict, global_mean_dict)
							valence_with_loss.append((vl,temp_LOSS))


					else:
						valence_with_loss.append((valence_list,ori_LOSS))
				print(time.time()-timestamp)
				timestamp = time.time()

		return valence_with_loss


	def tune_by_resonance(self, LOSS, res, global_nomalized_dict, global_sigma_dict, global_mean_dict):
		valence_with_loss = self.apply_resonance(res.super_atom_idx_list, 
												 res.sum_of_valence, 
												 LOSS, 
												 res,
												 global_nomalized_dict, 
												 global_sigma_dict, 
												 global_mean_dict)  #first run
		#print(valence_with_loss)
		check = {}
		#print(valence_with_loss)
		for vwl in valence_with_loss:
			if str(sorted(vwl[0])) not in check:
				check[str(sorted(vwl[0]))] = [vwl]
			else:
				if vwl not in check[str(sorted(vwl[0]))]:
					check[str(sorted(vwl[0]))].append(vwl)

		if res.resonance_order == "2":
			#valence_with_loss = random.sample(valence_with_loss, 100)
			print("length:",len(valence_with_loss))
			for vwl in valence_with_loss:
				prime_valence_with_loss = self.apply_resonance(res.super_atom_idx_list, 
															   vwl[0], 
															   vwl[1], 
															   res, 
															   global_nomalized_dict, 
															   global_sigma_dict, 
															   global_mean_dict)
				for p in prime_valence_with_loss:
					if str(sorted(p[0])) not in check:
						check[str(sorted(p[0]))] = [p]
					else:
						if p not in check[str(sorted(p[0]))]:
							check[str(sorted(p[0]))].append(p)

		possible_resonance = {}
		for k,v in check.items():
			avg_loss = sum(l[1] for l in v)/len(v)
			possible_resonance[avg_loss] = v

		key = sorted(possible_resonance.keys())[0]

		the_resonance_result = possible_resonance[key]
		avg_LOSS = sum([i[1] for i in the_resonance_result])/len(the_resonance_result)
		#one_sum_of_valence = the_resonance_result[0][0]
		resonance_valence_list = [i[0] for i in the_resonance_result]
		return avg_LOSS, the_resonance_result


In [4]:
def get_Oxidation_States(m_id,i, atom_pool):

    GFOS = GET_FOS()
    delta_X = 0.1
    tolerance_list = valid_t_dict[m_id]
    print(tolerance_list)
    corr_t = []
    ls = time.time()
        
    for t in tolerance_list:
        print(t)
        res = RESULT()
        res.mid = m_id
        TN = TUNE()
        #try:
        if True:
            GFOS.loss_loop(m_id, delta_X, t, tolerance_list, res)
            print(res.idx)
            temp_pair_info = spider_pair_length_with_CN_unnorm(res.sum_of_valence, res)

            #now, the matched dict is the global normalization normed dict. 
            loss = cal_loss_func_by_MAP(temp_pair_info, 
                                        global_normalized_normed_dict, 
                                        global_sigma_dict, 
                                        global_mean_dict)
            N_spec = len(res.species_uni_list)
            res.initial_vl = res.sum_of_valence
            
            if len(res.super_atom_idx_list) > 0:
                if res.resonance_flag:
                    avg_LOSS, the_resonance_result = TN.tune_by_resonance(loss,
                                                                          res, 
                                                                          global_normalized_normed_dict,
                                                                          global_sigma_dict, 
                                                                          global_mean_dict)
                    res.final_vl = the_resonance_result[0][0]
                    same_after_resonance = True if res.final_vl == res.initial_vl else False
                    res.sum_of_valence = res.final_vl
                    print("Here")

                if atom_pool == "super":
                    process_atom_idx_list = res.super_atom_idx_list
                if atom_pool == "link":
                    process_atom_idx_list = res.link_atom_idx_list
                if atom_pool == "all":
                    process_atom_idx_list = res.idx 

                LOSS, res.final_vl = TN.tune_by_redox_in_certain_range_by_MAP(process_atom_idx_list, 
                                                                              loss, 
                                                                              res.sum_of_valence,
                                                                              0,
                                                                              res,
                                                                              global_normalized_normed_dict,
                                                                              global_sigma_dict, 
                                                                              global_mean_dict)
                res.sum_of_valence = res.final_vl
                same_after_tunation = True if res.final_vl == res.initial_vl else False
                same_after_resonance = True
                
            else:
                res.final_vl = res.initial_vl
                same_after_tunation = True
                same_after_resonance = True
                LOSS = loss
                
            parameters = {m_id: [res.resonance_flag, same_after_tunation, same_after_resonance]}
            
            loss_value = 1**N_spec * LOSS
            corr_t.append((t,loss_value,res))
        #except:
        else:
            None

    #try:
    if True:
        chosen_one = sorted(corr_t, key = lambda x:x[1])[0]
        res = chosen_one[2]
        t = chosen_one[0]
        #parameters = [m_id, LOSS, res.final_vl]
        #for loop
        temp_pair_info_normed = spider_pair_length_with_CN_normed(res)
        temp_pair_info = spider_bond_length(res)

        single_result_dict_normed = {t:temp_pair_info_normed}
        single_result_dict = {t:temp_pair_info}

        OS_result_with_ele = sorted([(i,j) for i,j in zip(res.final_vl, res.elements_list)], key = lambda x :x[1])
        OS_result = [ij[0] for ij in OS_result_with_ele]

        normalized_single_result_info = normalization(single_result_dict_normed)
        parameters = [m_id, normalized_single_result_info, single_result_dict, OS_result]
        tc = time.time() - ls
        print("Got the Formal Oxidation State of the %sth structure %s in %s seconds."%(i,m_id,tc))
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        return parameters
    #except:
    else:
        parameters = [m_id, None, None, None]
        tc = time.time() - ls
        print("Failed to analyze the %sth structure %s in %s seconds."%(i,m_id,tc))
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        return parameters

In [42]:
36*5

180

In [7]:
s = time.time()
m_id = "mp-1228217.cif"
i = 0
atom_pool = "all"
parameter = get_Oxidation_States(m_id,i, atom_pool)
e = time.time()
print(e-s)

[1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.18, 1.19, 1.21, 1.23, 1.24]
1.1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.11
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.12
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.13
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.14
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.15
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.16
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.18
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
1.19
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Here
1.21
[0, 1, 2, 3, 4, 5, 6